# CCIE Topic guide 

* 1.2 Routing Concepts
    * 1.2.a Administrative distance
    * 1.2.b VRF-lite
    * 1.2.c Static routing
    * 1.2.d Policy Based Routing
    * 1.2.e VRF aware routing with any routing protocol
    * 1.2.f Route filtering with any routing protocol
    * 1.2.g Manual summarization with any routing protocol
    * 1.2.h Redistribution between any pair of routing protocols
    * 1.2.i Routing protocol authentication
    * 1.2.j Bidirectional Forwarding Detection

# Static Routing 


## IP Routing Processes 
* Packet routing steps 
    1. Routing - Find outgoing interface for a packet 
    2. switching - move packets between interfaces (Processe switch, Fast switching, CEF) 
    3. Encapsulation - rebuild the packet's L2 header

### Steps 1 : Routing process
1. Packet enters to the router 
2. Find the __Longest Prefix match__ algorithm to match a destination from routing table 
    * `sh ip route 1.2.3.4`
3. Find the outgoing interface using __Recursive Lookup__ 
    * `1.2.3.4 via 5.6.7.8`
    * `5.6.7.8 via 9.0.1.2`
    * `9.0.1.2 via 3.4.5.6`
    * `3.4.5.6 Directly connected F1/0`
4. A route can't be installed on the routing table untill it passes the recursive lookup process
    * to see the actual recursive lookup process use `sh ip cef` command 
5. If multiple longest prefix matches take place 
    * Same protocol 
        * Choose lowest metric, if metric are same then perform Load balancing (by default equal cost) 
        * Internal routes are preffered over external 
    * Different protocols
        * Choolse lowest AD
        
### Step 2 : Switchig process
1. __Switching path__ : moving packets between interfaces (PS, FS, CEF)
2. Load balancing in switching path, Per-flow/destination (default) , per-packet
3. verification 
    * `sh ip interface` 
    * `sh ip cef [address] [detail]
    * `sh ip cef exact-route [source] [destination]`
    
### Step 3: Encapsulation Process
1. build the L2 header 
    * header type depends on engress interface (e.g. Ethernet, PPP etc.) 
2. for multipoint interface L3-L2 resolution ins required 
    * Ethernet : ARP 
        * Check the ARP cache : `show ip arp` 
        * if ARP miss, bradcast the request : `debug arp`
    * mGRE DMVPN : NHRP
        * check NHRP mapping table : `sh ip nhrp`
        * if NHRP mapping miss, ask NHS : `debug nhrp` , `debug dmvpn` 
        
There are two ways to find next hop, `ip route NET-ID MASK NH-IP` 
1. Recurse to interface : find the recursvely interface that reaches `NH-IP` matches 
2. If multipoint resolve next-hop : find L2 address of `NH-IP`
3. if Point-to-point recursion is not needed 


## Reliable Static Routing 
* Only criteria to install a static route is the route-recursion process, however network status to the ingress interface is not known to the subjected router. 
* To bring more intelligence, IP-SLA is setup that sends monitoring probe to check remote end condition. 
![](pics/reliable_static.png)
In the above figure, the static route configuration of the routers are given below. R1 and R2 has floating static routes set up. 

```
R1#sh run | sec ip route 
ip route 192.168.2.0 255.255.255.0 12.0.0.2
ip route 192.168.2.0 255.255.255.0 13.0.0.3 10

R2#sh run | sec ip route 
ip route 192.168.1.0 255.255.255.0 12.0.0.1
ip route 192.168.1.0 255.255.255.0 23.0.0.3 10

R3#sh run | sec ip route 
ip route 192.168.1.0 255.255.255.0 13.0.0.1
ip route 192.168.2.0 255.255.255.0 23.0.0.2
```
Now if f0/0 interface of R1 goes down, R2 would be unware of this fact and hence it wont float its staic route. 

```
R1(config)#int f0/0
R1(config-if)#shut  
*Mar  1 00:20:02.719: %LINK-5-CHANGED: Interface FastEthernet0/0, changed state to administratively down
*Mar  1 00:20:03.719: %LINEPROTO-5-UPDOWN: Line protocol on Interface FastEthernet0/0, changed state to down
R1(config-if)#do sh ip route static
S    192.168.2.0/24 [10/0] via 13.0.0.3


R2#sh ip route static 
S    192.168.1.0/24 [1/0] via 12.0.0.1
```
`R1` has floates its egress interface but not `R2`. To resolve this issue, tie them to a __SLA aggrement__. 
```
!R2
conf t
    ip sla monitor 1
        !create a echo prob monitor (id 1)
        type echo protocol ipIcmpEcho 12.0.0.1 source-interface f0/0  
    exit
    
    !start the generator 
    ip sla monitor schedule 1 start-time now life forever 
    
    track 1 rtr 1  !start a track object (id=1) with rtr = SLA and monitor id = 1      
    exit
    ip route 192.168.1.0 255.255.255.0 12.0.0.1 track 1
    ip route 192.168.1.0 255.255.255.0 23.0.0.3 10 ! backup route with metric=10
```